In [1]:
import json
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from tensorflow.keras.layers import RepeatVector, TimeDistributed
import colorama
colorama.init()
from colorama import Fore, Style

In [2]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_class = len(labels)

In [3]:
num_class

12

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)



In [5]:
from keras.utils import to_categorical

# Convert labels to one-hot encoded format
training_labels = to_categorical(training_labels, num_class)

In [6]:
training_labels.shape

(84, 12)

In [7]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [8]:
from keras.layers import LSTM, RepeatVector,Reshape  
from keras.layers import Bidirectional, GlobalAveragePooling1D
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras import Model
from tensorflow.keras.layers import Layer
import keras.backend as K
from keras import initializers, regularizers, constraints
from keras.layers import *
from attention import Attention   

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Reshape((1, embedding_dim))) 
model.add(LSTM(16, activation='relu'))
model.add(RepeatVector(max_len)) 
model.add(LSTM(16, activation='relu'))
model.add(Dense(num_class, activation='softmax'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 reshape (Reshape)           (None, 1, 16)             0         
                                                                 
 lstm (LSTM)                 (None, 16)                2112      
                                                                 
 repeat_vector (RepeatVector  (None, 20, 16)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 16)                2

In [14]:
epochs = 100
history = model.fit(padded_sequences, training_labels, epochs=epochs)

Epoch 1/100
3/3 [==============================] - 0s 7ms/step - loss: 0.0833 - accuracy: 0.7857
Epoch 2/100
3/3 [==============================] - 0s 9ms/step - loss: 0.0821 - accuracy: 0.7857
Epoch 3/100
3/3 [==============================] - 0s 9ms/step - loss: 0.0806 - accuracy: 0.7976
Epoch 4/100
3/3 [==============================] - 0s 8ms/step - loss: 0.0798 - accuracy: 0.7976
Epoch 5/100
3/3 [==============================] - 0s 12ms/step - loss: 0.0787 - accuracy: 0.7976
Epoch 6/100
3/3 [==============================] - 0s 8ms/step - loss: 0.0777 - accuracy: 0.7976
Epoch 7/100
3/3 [==============================] - 0s 10ms/step - loss: 0.0768 - accuracy: 0.7976
Epoch 8/100
3/3 [==============================] - 0s 8ms/step - loss: 0.0760 - accuracy: 0.7976
Epoch 9/100
3/3 [==============================] - 0s 9ms/step - loss: 0.0753 - accuracy: 0.7976
Epoch 10/100
3/3 [==============================] - 0s 9ms/step - loss: 0.0748 - accuracy: 0.7976
Epoch 11/100
3/3 [=========

In [15]:
model.save("Fchat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder1.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: Fchat_model\assets


INFO:tensorflow:Assets written to: Fchat_model\assets


In [17]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)

def chat():
    # Load trained model
    model = keras.models.load_model('Fchat_model')

    # Load tokenizer object
    with open('tokenizer1.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # Load label encoder object
    with open('label_encoder1.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # Parameters
    max_len = 20

    booking_info = {}  # Store booking information

    # Greet the user
    print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, np.random.choice(data['intents'][0]['responses']))

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for intent in data['intents']:
            if intent['tag'] == tag:
                if intent['tag'] == 'book_hotel':
                    print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, np.random.choice(intent['responses']))
                    for question in intent['questions']:
                        answer = input(question + " ")
                        booking_info[question] = answer
                    booking_id = random.randint(1000, 9999)
                    booking_info['booking_id'] = booking_id
                    print("ChatBot: Your booking has been confirmed! Booking ID: {}".format(booking_id))
                    break
                elif intent['tag'] == 'cancel_booking':
                    print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, np.random.choice(intent['responses']))
                    booking_id = input(intent['questions'][0] + " ")
                    print("ChatBot: Your booking with ID {} has been canceled.".format(booking_id))
                    break
                elif intent['tag'] == 'modify_booking':
                    print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, np.random.choice(intent['responses']))
                    for question in intent['questions']:
                        answer = input(question + " ")
                        booking_info[question] = answer
                    booking_id = booking_info['Enter your booking ID:']
                    print("ChatBot: Your booking with ID {} has been modified.".format(booking_id))
                    break
                else:
                    print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, np.random.choice(intent['responses']))
                    break

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
ChatBot: hey, have a nice day
1/1 [==============================] - 0s 306ms/step
ChatBot: I.m Joana, your bot assistant
1/1 [==============================] - 0s 23ms/step
ChatBot: You can call me alix.
1/1 [==============================] - 0s 22ms/step
ChatBot: Let's get started with your hotel booking!
ChatBot: Your booking has been confirmed! Booking ID: 1094
1/1 [==============================] - 0s 23ms/step
ChatBot: I'm sorry to hear that. To cancel your booking, please provide the booking ID.
ChatBot: Your booking with ID 1094 has been canceled.
1/1 [==============================] - 0s 21ms/step
ChatBot: Sure, to modify your booking, please provide the booking ID and the updated details.
ChatBot: Sorry, the provided booking ID does not match. Please try again.
User: 